In [24]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selectorlib import Extractor
import requests
import json
import time

In [27]:
def search_amazon(item):

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.amazon.com')
    search_box = driver.find_element_by_id('twotabsearchtextbox').send_keys(item)
    search_button = driver.find_element_by_id("nav-search-submit-text").click()

    driver.implicitly_wait(5)

    try:
        num_page = driver.find_element_by_xpath('//*[@class="a-pagination"]/li[3]')
    except NoSuchElementException:
        num_page = driver.find_element_by_class_name('a-last').click()

    driver.implicitly_wait(3)

    url_list = []

    for i in range(int(num_page.text)):
        page_ = i + 1
        url_list.append(driver.current_url)
        driver.implicitly_wait(4)
        click_next = driver.find_element_by_class_name('a-last').click()
        print("Page " + str(page_) + " grabbed")

    driver.quit()


    with open('search_results_urls.txt', 'w') as filehandle:
        for result_page in url_list:
            filehandle.write('%s\n' % result_page)

    print("---DONE---")

In [28]:
def scrape(url):

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create
    return e.extract(r.text)

search_amazon('microban') # <------ search query goes here.

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\M1060555\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
Page 1 grabbed
Page 2 grabbed
---DONE---


In [29]:
# courtesy of https://github.com/scrapehero-code/amazon-scraper
# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('search_results.yml')

# product_data = []
with open("search_results_urls.txt",'r') as urllist, open('search_results_output.jsonl','w') as outfile:
    for url in urllist.read().splitlines():
        data = scrape(url)
        if data:
            for product in data['products']:
                product['search_url'] = url
                print("Saving Product: %s"%product['title'].encode('utf8'))
                json.dump(product,outfile)
                outfile.write("\n")
                # sleep(5)


Saving Product: b'Microban 24 Hour Disinfectant Sanitizing Spray, Citrus Scent, 2 Count, 15 fl oz Each'
Saving Product: b'Microban 24 Hour Sanitizing Spray, Fresh Scent 12.5 OZ'
Saving Product: b'Microban Professional Sanitizing Spray'
Saving Product: b'Microban 24 Hour Multi-Purpose Cleaner, Sanitizing and Disinfectant Spray, Fresh Scent, 22 Ounce (Pack of 2)'
Saving Product: b'Microban Sanitizing Spray (Fresh Scent)'
Saving Product: b'Microban 24 Hour Bathroom Cleaner and Sanitizing Spray, Fresh Scent - 32 Ounce (Pack of 2)'
Saving Product: b'Microban 24 Hour Disinfectant Sanitizing Spray, Citrus Scent'
Saving Product: b'Microban 24 Hour Bathroom Cleaner and Sanitizing Spray, Citrus Scent - 32 Ounce (Pack of 2)'
Saving Product: b'Microban 24 Hour Bathroom Cleaner and Sanitizing Spray, Citrus Scent - 32 Ounce (Pack of 3)'
Saving Product: b'Microban 24 Hour Bathroom Cleaner and Sanitizing Spray, Citrus Scent - 32 Ounce (Pack of 4)'
Saving Product: b'MSAAEX 50 Pcs Disposable 4-ply Non-W